# Joe's code place

![Hackerman](https://i.kym-cdn.com/entries/icons/mobile/000/021/807/ig9OoyenpxqdCQyABmOQBZDI0duHk2QZZmWg2Hxd4ro.jpg)

# Code

In [169]:
# Imports
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [170]:
""" Ticker Library init
portfolio = {'Sector':[],'Company Name':[],'Ticker':[]}
hello = [
    ['Technology','Netflix','NFLX'],
    ['Technology','Meta','FB'],
    ['Technology','Uber','UBER'],
    ['Technology','Microchip Technology','MCHP'],
    ['Technology','AirBnB','ABNB'],
    ['Energy','Diamondback Energy Inc.','FANG'],
    ['Energy','Marathon Oil Corp.','MRO'],
    ['Energy','Devon Energy Corp.','DVN'],
    ['Energy','SunPower Corp.','SPWR'],
    ['Energy','Renewable Energy Group Inc.','REGI'],
    ['Finance','McKinsey & Company','MTRX'],
    ['Finance','BlackRock','BLK'],
    ['Finance','PayPal','PYPL'],
    ['Finance','Mercadolibre Inc','MELI'],
    ['Finance','SoFi','SOFI']
]
for row in hello:
    portfolio['Sector'].append(row[0])
    portfolio['Company Name'].append(row[1])
    portfolio['Ticker'].append(row[2])
portfolio = pd.DataFrame(portfolio).set_index('Sector')
portfolio.to_csv('./Data/Cleaned_Data/Ticker_library.csv')
"""

" Ticker Library init\nportfolio = {'Sector':[],'Company Name':[],'Ticker':[]}\nhello = [\n    ['Technology','Netflix','NFLX'],\n    ['Technology','Meta','FB'],\n    ['Technology','Uber','UBER'],\n    ['Technology','Microchip Technology','MCHP'],\n    ['Technology','AirBnB','ABNB'],\n    ['Energy','Diamondback Energy Inc.','FANG'],\n    ['Energy','Marathon Oil Corp.','MRO'],\n    ['Energy','Devon Energy Corp.','DVN'],\n    ['Energy','SunPower Corp.','SPWR'],\n    ['Energy','Renewable Energy Group Inc.','REGI'],\n    ['Finance','McKinsey & Company','MTRX'],\n    ['Finance','BlackRock','BLK'],\n    ['Finance','PayPal','PYPL'],\n    ['Finance','Mercadolibre Inc','MELI'],\n    ['Finance','SoFi','SOFI']\n]\nfor row in hello:\n    portfolio['Sector'].append(row[0])\n    portfolio['Company Name'].append(row[1])\n    portfolio['Ticker'].append(row[2])\nportfolio = pd.DataFrame(portfolio).set_index('Sector')\nportfolio.to_csv('./Data/Cleaned_Data/Ticker_library.csv')\n"

## Twitter

In [171]:
import tweepy

In [172]:
def init_twitter_API():
    return tweepy.API(
        tweepy.OAuth1UserHandler(
            # Generated when you setup your Twitter Dev account
            os.getenv("twitter_api_consumer_key"),
            os.getenv("twitter_api_consumer_secret_key"),
            # You have to generate the below after you setup your Twitter Dev Account
            os.getenv("twitter_api_access_key"),
            os.getenv("twitter_api_access_secret_key")
        )
    )

# tw = init_twitter_API()

## Python Edgar

In [173]:
tickers = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')['Ticker'].to_list()

In [174]:
from sec_edgar_downloader import Downloader

In [175]:
def create_SEC(path,tickers,filing_type,after_date,before_date=None):
    dl = Downloader(f'{path}')
    for ticker in tickers:
        dl.get(filing_type, ticker, after=after_date,before=before_date)

In [176]:
# Uncomment to download
# create_SEC('./Data/Raw_Data/',tickers,"10-Q",after_date='2012-06-01')

In [177]:
def txt_finder():
    txts = []
    for root, dirs, files in os.walk(".", topdown=False):
        for name in files:
            os.path.join(root, name)
            if name[-4:]=='.txt':
                txts.append(f'{root}/{name}')
        for name in dirs:
            os.path.join(root, name)
    return txts
sec_paths = txt_finder()

In [178]:
def path_part(path,before,after):
    beforeNum = path.find(before)+len(before)
    afterNum = path.find(after)
    return path[beforeNum:afterNum]

def sec_df_from_paths(sec_paths,filing_type):
    data = {
        'Ticker':[],
        'DocName':[],
        'DocPath':[],
        'FilingType':[],
        'DocText':[]
    }
    for sec in sec_paths:
        data['Ticker'].append(path_part(sec,'sec-edgar-filings\\',f'\\{filing_type}'))
        data['DocName'].append(path_part(sec,f'\\{filing_type}\\','/full-submission'))
        data['DocPath'].append(sec)
        data['FilingType'].append(filing_type)
        sec_txt = open(sec, "r")
        data['DocText'].append(sec_txt.read())
        sec_txt.close()
    return pd.DataFrame(data)

df_10Q = sec_df_from_paths(sec_paths,'10-Q')

In [179]:
df_10Q.head().tail(2)

,Ticker,DocName,DocPath,FilingType,DocText
3,ABNB,0001628280-21-016979,.\Data\Raw_Data\sec-edgar-filings\ABNB\10-Q\00...,10-Q,<SEC-DOCUMENT>0001628280-21-016979.txt : 20210...
4,BLK,0001193125-12-343728,.\Data\Raw_Data\sec-edgar-filings\BLK\10-Q\000...,10-Q,<SEC-DOCUMENT>0001193125-12-343728.txt : 20120...


In [180]:
import re
def restore_windows_1252_characters(restore_string):
    """
        Replace C1 control characters in the Unicode string s by the
        characters at the corresponding code points in Windows-1252,
        where possible.
    """

    def to_windows_1252(match):
        try:
            return bytes([ord(match.group(0))]).decode('windows-1252')
        except UnicodeDecodeError:
            # No character at the corresponding code point: remove it.
            return ''
        
    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

In [181]:
# df_10Q['DocText'].apply(restore_windows_1252_characters)
df_10Q_test = df_10Q.head()
df_10Q_test['DocText'].apply(restore_windows_1252_characters)

0    <SEC-DOCUMENT>0001559720-21-000017.txt : 20211...
1    <SEC-DOCUMENT>0001559720-22-000009.txt : 20220...
2    <SEC-DOCUMENT>0001628280-21-010389.txt : 20210...
3    <SEC-DOCUMENT>0001628280-21-016979.txt : 20210...
4    <SEC-DOCUMENT>0001193125-12-343728.txt : 20120...
Name: DocText, dtype: object

In [182]:
df_10Q['DocText'].loc[0][:1000]

'<SEC-DOCUMENT>0001559720-21-000017.txt : 20211105\n<SEC-HEADER>0001559720-21-000017.hdr.sgml : 20211105\n<ACCEPTANCE-DATETIME>20211105163210\nACCESSION NUMBER:\t\t0001559720-21-000017\nCONFORMED SUBMISSION TYPE:\t10-Q\nPUBLIC DOCUMENT COUNT:\t\t70\nCONFORMED PERIOD OF REPORT:\t20210930\nFILED AS OF DATE:\t\t20211105\nDATE AS OF CHANGE:\t\t20211105\n\nFILER:\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tAirbnb, Inc.\n\t\tCENTRAL INDEX KEY:\t\t\t0001559720\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tSERVICES-TO DWELLINGS & OTHER BUILDINGS [7340]\n\t\tIRS NUMBER:\t\t\t\t263051428\n\t\tSTATE OF INCORPORATION:\t\t\tDE\n\t\tFISCAL YEAR END:\t\t\t1231\n\n\tFILING VALUES:\n\t\tFORM TYPE:\t\t10-Q\n\t\tSEC ACT:\t\t1934 Act\n\t\tSEC FILE NUMBER:\t001-39778\n\t\tFILM NUMBER:\t\t211385075\n\n\tBUSINESS ADDRESS:\t\n\t\tSTREET 1:\t\t888 BRANNAN ST.\n\t\tCITY:\t\t\tSAN FRANCISCO\n\t\tSTATE:\t\t\tCA\n\t\tZIP:\t\t\t94103\n\t\tBUSINESS PHONE:\t\t415.800.5959\n\n\tMAIL ADDRESS:\t\n\t\tSTREET 1:\t\

In [183]:
from sec_api import ExtractorApi
sec_api = os.getenv("sec_api_key")
extractorApi = ExtractorApi(sec_api)

# Tesla 10-K filing
filing_url = "https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.htm"

# get the standardized and cleaned text of section 1A "Risk Factors"
# section_text = extractorApi.get_section(filing_url, "1A", "text")

# print(section_text)

In [184]:
len("CONFORMED PERIOD OF REPORT:\t")

28

In [185]:
def DocText_extraction(text,before,after):
    return text[
        text.find(before)+len(before):
        text.find(after)
        ]

In [186]:
def SEC_date(DocText):
    """Returns Conformed Period of Report when given the text of an SEC report"""
    return DocText_extraction(DocText,"CONFORMED PERIOD OF REPORT:\t","\nFILED AS OF DATE:")
def SEC_CIK(DocText):
    """Returns CENTRAL INDEX KEY when given the text of an SEC report"""
    return DocText_extraction(DocText,"CENTRAL INDEX KEY:\t\t\t","\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:")
    

In [187]:
def extract_all_SEC_dates(df:pd.DataFrame):
    df['SECdates'] = df['DocText'].apply(SEC_date)
def extract_all_SEC_CIK(df:pd.DataFrame):
    df['SEC_CIK'] = df['DocText'].apply(SEC_CIK)
extract_all_SEC_dates(df_10Q)
extract_all_SEC_CIK(df_10Q)
df_10Q.head()

,Ticker,DocName,DocPath,FilingType,DocText,SECdates,SEC_CIK
0,ABNB,0001559720-21-000017,.\Data\Raw_Data\sec-edgar-filings\ABNB\10-Q\00...,10-Q,<SEC-DOCUMENT>0001559720-21-000017.txt : 20211...,20210930,0001559720
1,ABNB,0001559720-22-000009,.\Data\Raw_Data\sec-edgar-filings\ABNB\10-Q\00...,10-Q,<SEC-DOCUMENT>0001559720-22-000009.txt : 20220...,20220331,0001559720
2,ABNB,0001628280-21-010389,.\Data\Raw_Data\sec-edgar-filings\ABNB\10-Q\00...,10-Q,<SEC-DOCUMENT>0001628280-21-010389.txt : 20210...,20210331,0001559720
3,ABNB,0001628280-21-016979,.\Data\Raw_Data\sec-edgar-filings\ABNB\10-Q\00...,10-Q,<SEC-DOCUMENT>0001628280-21-016979.txt : 20210...,20210630,0001559720
4,BLK,0001193125-12-343728,.\Data\Raw_Data\sec-edgar-filings\BLK\10-Q\000...,10-Q,<SEC-DOCUMENT>0001193125-12-343728.txt : 20120...,20120630,0001364742


In [197]:
def url_maker(df:pd.DataFrame):
    #Example:
    # "https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.htm"
    # url_base:                 https://www.sec.gov/Archives/edgar/data/
    # SEC CIK:                  1318605
    # SEC DOCName:              000156459021004599
    # Ticker:                   tsla
    # Doc Type:                 10k
    # Date (date 2020-02-31):   20201231 


    url_base = "https://www.sec.gov/Archives/edgar/data/"
    SEC_cik = df['SEC_CIK']
    SEC_DocName = df['DocName'].apply(lambda x: re.sub("-","",x))
    ticker = df['Ticker'].apply(str.lower)
    doc_type = df['FilingType'].apply(lambda x: re.sub("-","",x))
    date = df['SECdates']
    return url_base + SEC_cik + "/" + SEC_DocName + "/" + ticker + "-" + doc_type + "_" + date + '.htm'

urls = url_maker(df_10Q)
df_10Q['urls'] = urls
for x in urls:
    print(x)

https://www.sec.gov/Archives/edgar/data/0001559720/000155972021000017/abnb-10Q_20210930.htm
https://www.sec.gov/Archives/edgar/data/0001559720/000155972022000009/abnb-10Q_20220331.htm
https://www.sec.gov/Archives/edgar/data/0001559720/000162828021010389/abnb-10Q_20210331.htm
https://www.sec.gov/Archives/edgar/data/0001559720/000162828021016979/abnb-10Q_20210630.htm
https://www.sec.gov/Archives/edgar/data/0001364742/000119312512343728/blk-10Q_20120630.htm
https://www.sec.gov/Archives/edgar/data/0001364742/000119312512460708/blk-10Q_20120930.htm
https://www.sec.gov/Archives/edgar/data/0001364742/000119312513210506/blk-10Q_20130331.htm
https://www.sec.gov/Archives/edgar/data/0001364742/000119312513326890/blk-10Q_20130630.htm
https://www.sec.gov/Archives/edgar/data/0001364742/000119312513434891/blk-10Q_20130930.htm
https://www.sec.gov/Archives/edgar/data/0001364742/000119312514192207/blk-10Q_20140331.htm
https://www.sec.gov/Archives/edgar/data/0001364742/000119312514302181/blk-10Q_20140630

In [198]:
df_10Q.head()

,Ticker,DocName,DocPath,FilingType,DocText,SECdates,SEC_CIK,urls
0,ABNB,0001559720-21-000017,.\Data\Raw_Data\sec-edgar-filings\ABNB\10-Q\00...,10-Q,<SEC-DOCUMENT>0001559720-21-000017.txt : 20211...,20210930,0001559720,https://www.sec.gov/Archives/edgar/data/000155...
1,ABNB,0001559720-22-000009,.\Data\Raw_Data\sec-edgar-filings\ABNB\10-Q\00...,10-Q,<SEC-DOCUMENT>0001559720-22-000009.txt : 20220...,20220331,0001559720,https://www.sec.gov/Archives/edgar/data/000155...
2,ABNB,0001628280-21-010389,.\Data\Raw_Data\sec-edgar-filings\ABNB\10-Q\00...,10-Q,<SEC-DOCUMENT>0001628280-21-010389.txt : 20210...,20210331,0001559720,https://www.sec.gov/Archives/edgar/data/000155...
3,ABNB,0001628280-21-016979,.\Data\Raw_Data\sec-edgar-filings\ABNB\10-Q\00...,10-Q,<SEC-DOCUMENT>0001628280-21-016979.txt : 20210...,20210630,0001559720,https://www.sec.gov/Archives/edgar/data/000155...
4,BLK,0001193125-12-343728,.\Data\Raw_Data\sec-edgar-filings\BLK\10-Q\000...,10-Q,<SEC-DOCUMENT>0001193125-12-343728.txt : 20120...,20120630,0001364742,https://www.sec.gov/Archives/edgar/data/000136...


All 10-K and 10-Q sections can be extracted:

    1 - Business
    1A - Risk Factors
    1B - Unresolved Staff Comments
    2 - Properties
    3 - Legal Proceedings
    4 - Mine Safety Disclosures
    5 - Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities
    6 - Selected Financial Data (prior to February 2021)
    7 - Management’s Discussion and Analysis of Financial Condition and Results of Operations
    7A - Quantitative and Qualitative Disclosures about Market Risk
    8 - Financial Statements and Supplementary Data
    9 - Changes in and Disagreements with Accountants on Accounting and Financial Disclosure
    9A - Controls and Procedures
    9B - Other Information
    10 - Directors, Executive Officers and Corporate Governance
    11 - Executive Compensation
    12 - Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters
    13 - Certain Relationships and Related Transactions, and Director Independence
    14 - Principal Accountant Fees and Services
> https://pypi.org/project/sec-api/
